In [2]:
# Import the required libraries for webscraping
from bs4 import BeautifulSoup
import requests
import re
# Import requried utility libraries
import os

### Get urls from the input folder.
It is assumed that the input folder is at the same level with src, and the file holding the urls is called from_urls. Also it is assumed that the urls are stored and separated by newlines in the input file.\
It is also assumed that only the first two lines of the input file will be valid URLs. No verificiation is carried out for the input data as it is outside the assignment scope


In [ ]:
# Initialize the url list
url_dict = {
    'season_126': '',
    'season_womens_seven': ''
}

# Define the import file path
INPUT_FILE_NAME = 'from_urls'

url_file_fn = os.path.join(
    os.path.abspath('..'), 
    'input',
    INPUT_FILE_NAME
)

# Open the file and read the urls into the url list
url_file = open(url_file_fn, 'r')
for k, v in url_dict.items():
    line = url_file.readline()
    if line:
        url_dict[k] = line.strip()
    else:
        pass
url_file.close()

### Request data from Wikipedia for AFL Season 126 and AFL Women's Season Seven
The raw data will also be saved into a dictionary of two BeautifulSoup objects

In [ ]:
# Initialize the BeautifulSoup object dictionary
bs_dict = {
    'season_126': None,
    'season_womens_seven': None
}

# Iterate through url_dict and save the BeautifulSoup object to bs_dict
for k, v in url_dict.items():
    response = requests.get(v)
    html = response.text
    bs_dict[k] = BeautifulSoup(html, 'html.parser')
    response.close()

bs_dict

### Extract from the BeautifuleSoup objects for the tables under "Home-and-away season"

In [16]:
# TO-DO: Iterate through the BeautifulSoup object and extract all the tables under 
# body -> div class mw-page-container -> div class mw-content-container
# -> div class vector-body ve-init-mw-desktopArticleTarget-targetContainer
# -> div class mw-body-content mw-content-ltr
# -> div class mw-parser-output -> ALL non-"wikitable" class tables
# Hint: RegEx might be used to filter out the "wikitable" class tables
# and all these tables objects can be saved to a list

match_list = ['foo']*100

### Extract the information from the match_list and save to a list of strings

In [17]:
# TO-DO: iterate through the match_list and extract the following information
"""
Round Number (example: "1")
Day of the game (example: "Friday")
Date of the game (example: "19-Aug")
Time of the game (example: "7:50 pm")
First team name (example: "Brisbane Lions")
First team score in points only (example: "57")
First team win status (either "def. by" or "def.")
Second team name (example: "Melbourne")
Second team score in points only (example: "115")
Game location (example: "The Gabba")
Stadium Attendees (example: "32172")
"""
# then save them to a list of strings, separated by comma
# each match should be a new line
COLUMN_NAMES = ','.join([
    'Round Number',
    'Day of the game',
    'Date of the game',
    'Time of the game',
    'First team name',
    'First team score in points only',
    'First team win status',
    'Second team name',
    'Second team score in points only',
    'Game location',
    'Stadium Attendees'
])
result_list = [COLUMN_NAMES]

for match in match_list:
    result_str = 'bar'
    result_list.append(result_str)

### Save the result to a csv output
It is assumed that the output folder is at the same level as the src folder \
It is also assumed that a csv file output named scraped_match_table.csv file will exist and contain the result \
The file will be overwritten regardless of its existing contents and no verification of existence will be carried out\
since it is outside the assignment scope

In [18]:
# Define the output file path
OUTPUT_FILE_NAME = 'scraped_match_table.csv'
scraped_match_table_fn = os.path.join(
    os.path.abspath('..'), 
    'output',
    OUTPUT_FILE_NAME
)

# Write the result_list to the output file
result_file = open(scraped_match_table_fn, 'w')
for result_str in result_list:
    result_file.write(f"{result_str}\n")
result_file.close()